<a href="https://colab.research.google.com/github/marcinkus017/Test/blob/main/Gratka_pl_webscrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Libraries and selenium setup

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import requests as r
import json
import time #to make Python wait, so the webpage can load fully
import numpy as np
import pandas as pd
from urllib.parse import urlparse #needed for url generation in the Linked in URL generator phase
from selenium.webdriver.common.keys import Keys #needed for inserting the email and password
from selenium.webdriver.common.by import By #needed for selecting the elements of the webpage
from random import randint #to generate random ID, if ID is missing

##Create random User Agent (to make gratka think that it is call from another browser)

In [ ]:
!pip install fake-useragent
from fake_useragent import UserAgent

ua = UserAgent()
userAgent = ua.random
print(userAgent)

Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36


##Set up selenium wd

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#chrome_options.add_argument('--proxy-server=%s' % PROXY) #for proxy
chrome_options.add_argument(f'user-agent={userAgent}')
#chrome_options.add_argument("user-data-dir=selenium") #for cookies - https://stackoverflow.com/questions/15058462/how-to-save-and-load-cookies-using-python-selenium-webdriver
wd = webdriver.Chrome(options=chrome_options) #wd = webdriver.Chrome('chromedriver', options=chrome_options)

#Test on 1 search

##Call the page

In [ ]:
#url = 'https://gratka.pl/nieruchomosci/mieszkania?page=4&pietro:min=1&powierzchnia-w-m2:min=60&liczba-pokoi:min=3&cena-za-m2:min=1&powierzchnia-w-m2:max=100&lokalizacja[0]=117413&lokalizacja[1]=116271&lokalizacja[2]=123253&lokalizacja[3]=123759&lokalizacja[4]=123251&lokalizacja[5]=113731&lokalizacja[6]=119523&lokalizacja[7]=122201&lokalizacja[8]=119515&lokalizacja[9]=113733&sort=cheap'
# url = 'https://gratka.pl/nieruchomosci/mieszkania?' + '?page=' + 1 + "&pietro:min=' + 1 + '&powierzchnia-w-m2:min=' + 60 + '&powierzchnia-w-m2:max=' + 120 + "&liczba-pokoi:min=" + 3 + '&cena-za-m2:min=' + 1
# https://gratka.pl/nieruchomosci/mieszkania/poznan?pietro:min=1&powierzchnia-w-m2:min=60&liczba-pokoi:min=3&cena-za-m2:min=1&powierzchnia-w-m2:max=100

In [ ]:
url = 'https://gratka.pl/nieruchomosci/mieszkania/'+ 'poznan' + '?page=' + str(1) + '&pietro:min=' + str(1) + '&powierzchnia-w-m2:min=' + str(60) + '&powierzchnia-w-m2:max=' + str(120) + '&liczba-pokoi:min=' + str(3) + '&cena-za-m2:min=' + str(1)

# get number of pages to be scrapped
# open the url
wd.get(url)

# get element and then click the element (agreeing the cookies storage). If that's not possible (the cookies pop up didn't pop up) then do nothing
try:
  element = wd.find_element(By.XPATH, "//button[@class='cmp-button_button cmp-intro_acceptAll ']")
  element.click() # click the element
except:
  pass

pagination = wd.find_element(By.CLASS_NAME, "pagination")
last_page = pagination.find_elements(By.TAG_NAME, "a")
last_page = last_page[-2].text #not -1, because there is the last button with an arrow (next_page), not number of last page
print(last_page)

26


In [ ]:
df_listings_raw = pd.DataFrame()

pietro_min            = 0
powierzchnia_w_m2_min = 60
powierzchnia_w_m2_max = 120
liczba_pokoi_min      = 3
cena_za_m2_min        = 1

for page in range(1, int(last_page)+1):

  url = 'https://gratka.pl/nieruchomosci/mieszkania/'+ 'poznan' + '?page=' + str(page)
  + '&pietro:min='            + str(pietro_min)
  + '&powierzchnia-w-m2:min=' + str(powierzchnia_w_m2_min)
  + '&powierzchnia-w-m2:max=' + str(powierzchnia_w_m2_max)
  + '&liczba-pokoi:min='      + str(liczba_pokoi_min)
  # + '&cena-za-m2:min='        + str(cena_za_m2_min)

  # open the url
  try:
    wd.get(url)
  except:
    print("Some issue encountered while trying to read the webpage. Restarting the webdriver and trying again.")
    wd = webdriver.Chrome(options=chrome_options) #wd = webdriver.Chrome('chromedriver', options=chrome_options)
    wd.get(url)

  time.sleep(5)

  # get element and then click the element (agreeing the cookies storage). If that's not possible (the cookies pop up didn't pop up) then do nothing
  try:
    element = wd.find_element(By.XPATH, "//button[@class='cmp-button_button cmp-intro_acceptAll ']")
    element.click() # click the element
  except:
    pass

  #link and title
  href = []
  title = []
  listings = wd.find_elements(By.CLASS_NAME, "teaserLink")
  for listing in listings:
    href.append(listing.get_attribute("href"))
    title.append(listing.get_attribute("title"))

  df_listings = pd.DataFrame({"Link": href, "Tytul": title})
  df_listings["Tytul_lok"] = df_listings["Tytul"].replace(dict.fromkeys(['Nowe ', 'nowe ', 'mieszkanie ', 'Mieszkanie ', 'Poznań ', 'Poznań, '], ''), regex=True)
  df_listings["Tytul_dzielnica"] = df_listings["Tytul_lok"].str.split(', ').str[0]
  df_listings["Tytul_ulica"] = df_listings["Tytul_lok"].str.split(', ').str[1]
  df_listings = df_listings.drop('Tytul_lok', axis=1)

  #location
  location = []
  listings = wd.find_elements(By.CLASS_NAME, "teaserUnified__location")
  for listing in listings:
    location.append(listing.text.split(','))

  #df_listings['Wojewodztwo'] = pd.DataFrame(item[2] for item in location)
  df_listings['Miasto'] = pd.DataFrame(item[0] for item in location)
  df_listings['Dzielnica'] = pd.DataFrame(item[1] for item in location)

  #params
  params = []
  listings = wd.find_elements(By.CLASS_NAME, "teaserUnified__params")
  for listing in listings:
    params.append(listing.text.splitlines())
    # print(listing.get_attribute("title"))

  #params = [str(i).replace( ' m2', '') for i in params] #strip those name out of " m2" suffix
  #params = [str(i).replace( ',', '.') for i in params] #replace the european comma with american dot and format as number
  #params = [str(i).replace( ' pokoje', '') for i in params] #strip those name out of " pokoje" suffix

  df_listings['Metraz'] = pd.DataFrame(item[0] for item in params)
  df_listings['Metraz'] = df_listings['Metraz'].str.replace(' m2', '').str.replace(',', '.').astype(float)
  df_listings['Pokoje'] = pd.DataFrame(item[1] for item in params)
  df_listings['Pokoje'] = df_listings['Pokoje'].replace(dict.fromkeys([' pokoje', ' pokoi', 'więcej niż '], ''), regex=True).astype(int)

  #price
  price = []
  listings = wd.find_elements(By.CLASS_NAME, "teaserUnified__price")
  for listing in listings:
    price.append(listing.text.splitlines())

  for item in price:
    if item[0] == 'Zapytaj o cenę': item.append(np.NAN) #if the entry has "Zapytaj o cenę" label it doesn't have any price, so the NAN is inserted

  df_listings['Cena'] = pd.DataFrame(item[0].replace(' zł', '').replace(' ', '') for item in price)
  df_listings['Cena_za_m2'] = pd.DataFrame(item[1] for item in price) #.replace(' zł/m2', '') #.str[:-6]

  #ID
  nr_ref = []
  listings = wd.find_elements(By.CLASS_NAME, "teaserUnified__details")
  for listing in listings:
    nr_ref.append(listing.text.splitlines())

  nr_ref_clean = []

  for item in nr_ref:
    temp = len(nr_ref_clean)
    for subitem in item: #for each subitem of the item of the list
      if subitem[:15] == 'Nr ref. gratka-':
        nr_ref_clean.append(subitem[15:]) #for the item with the ID present (one with prefix) get only the ID without the prefix and add it to the list
    if len(nr_ref_clean) != temp+1: #if no new entries have been added for a given item (no ID detected for a posting)
        nr_ref_clean.append("NoID-" + str(randint(1, 1000000))) # generate some numbers, so that this column doesn't have duplicates (with 1:100000 chance, being reasonable low chance)
  df_listings['ID'] = nr_ref_clean
  # nr_ref = [item for sublist in nr_ref for item in sublist] #flatten a list of lists to just a single list
  # nr_ref = [c for c in nr_ref if "Nr ref. gratka-" in c] #get only elements which are containg an ID of a listing #or nivea_comments = filter(lambda c: "nivea" in c, comments)
  # nr_ref = [str(i).replace( 'Nr ref. gratka-', '') for i in nr_ref_clean] #strip those name out of "Nr ref. gratka-" prefix

  print('Page just scrapped: ' + str(page))

  df_listings_raw = pd.concat([df_listings_raw, df_listings])

Page just scrapped: 1
Page just scrapped: 2
Page just scrapped: 3
Page just scrapped: 4
Page just scrapped: 5
Page just scrapped: 6
Page just scrapped: 7
Page just scrapped: 8
Page just scrapped: 9
Page just scrapped: 10
Page just scrapped: 11
Page just scrapped: 12
Page just scrapped: 13
Page just scrapped: 14
Page just scrapped: 15
Page just scrapped: 16
Page just scrapped: 17
Page just scrapped: 18
Page just scrapped: 19
Page just scrapped: 20
Page just scrapped: 21
Page just scrapped: 22
Page just scrapped: 23
Page just scrapped: 24
Page just scrapped: 25
Page just scrapped: 26


In [ ]:
df_listings_raw

,Link,Tytul,Tytul_dzielnica,Tytul_ulica,Miasto,Dzielnica,Metraz,Pokoje,Cena,Cena_za_m2,ID
0,https://gratka.pl/nieruchomosci/mieszkanie-poz...,"Mieszkanie Poznań Sołacz, ul. Piątkowska",Sołacz,ul. Piątkowska,Poznań,Sołacz,73.34,3,733000,9 995 zł/m2,469460201
1,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Jeżyce, ul. Dąbrowskiego",Jeżyce,ul. Dąbrowskiego,Poznań,Jeżyce,81.09,3,Zapytajocenę,NaN,10115-F/12
2,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Grunwald, ul. Marcelińs...",Grunwald,ul. Marcelińska 18,Poznań,Grunwald,62.51,3,Zapytajocenę,NaN,10562-B2-60
3,https://gratka.pl/nieruchomosci/na-sprzedaz-sl...,Na sprzedaż słoneczne i ciche mieszkanie na os...,Na sprzedaż słoneczne i ciche na os. B Chrobre...,NaN,Poznań,Piątkowo,73.60,5,629000,8 546 zł/m2,NoID-283673
4,https://gratka.pl/nieruchomosci/rezerwacja-prz...,REZERWACJA Przestronne mieszkanie 3 pokojowe n...,REZERWACJA Przestronne 3 pokojowe na Łazarzu 7...,NaN,Poznań,Łazarz,78.00,3,589000,7 551 zł/m2,NoID-406487
...,...,...,...,...,...,...,...,...,...,...,...
10,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Strzeszyn, ul. Literacka",Strzeszyn,ul. Literacka,Poznań,Strzeszyn,65.92,3,606464,9 200 zł/m2,9161-2_M15
11,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Strzeszyn, ul. Literacka",Strzeszyn,ul. Literacka,Poznań,Strzeszyn,70.43,4,647956,9 200 zł/m2,9161-1_M9
12,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Strzeszyn, ul. Literacka",Strzeszyn,ul. Literacka,Poznań,Strzeszyn,66.00,3,607200,9 200 zł/m2,9161-2_M6
13,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Strzeszyn, ul. Literacka",Strzeszyn,ul. Literacka,Poznań,Strzeszyn,66.47,4,611524,9 200 zł/m2,9161-4_M13


In [ ]:
test = df_listings_raw[df_listings_raw['Tytul_ulica'].notnull()]
test

,Link,Tytul,Tytul_dzielnica,Tytul_ulica,Miasto,Dzielnica,Metraz,Pokoje,Cena,Cena_za_m2,ID
0,https://gratka.pl/nieruchomosci/mieszkanie-poz...,"Mieszkanie Poznań Sołacz, ul. Piątkowska",Sołacz,ul. Piątkowska,Poznań,Sołacz,73.34,3,733000,9 995 zł/m2,469460201
1,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Jeżyce, ul. Dąbrowskiego",Jeżyce,ul. Dąbrowskiego,Poznań,Jeżyce,81.09,3,Zapytajocenę,NaN,10115-F/12
2,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Grunwald, ul. Marcelińs...",Grunwald,ul. Marcelińska 18,Poznań,Grunwald,62.51,3,Zapytajocenę,NaN,10562-B2-60
5,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Grunwald, ul. Marcelińs...",Grunwald,ul. Marcelińska 18,Poznań,Grunwald,61.77,3,Zapytajocenę,NaN,10562-B2-22
6,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Jeżyce, ul. Dąbrowskiego",Jeżyce,ul. Dąbrowskiego,Poznań,Jeżyce,83.21,4,Zapytajocenę,NaN,10115-F/8
...,...,...,...,...,...,...,...,...,...,...,...
10,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Strzeszyn, ul. Literacka",Strzeszyn,ul. Literacka,Poznań,Strzeszyn,65.92,3,606464,9 200 zł/m2,9161-2_M15
11,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Strzeszyn, ul. Literacka",Strzeszyn,ul. Literacka,Poznań,Strzeszyn,70.43,4,647956,9 200 zł/m2,9161-1_M9
12,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Strzeszyn, ul. Literacka",Strzeszyn,ul. Literacka,Poznań,Strzeszyn,66.00,3,607200,9 200 zł/m2,9161-2_M6
13,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Strzeszyn, ul. Literacka",Strzeszyn,ul. Literacka,Poznań,Strzeszyn,66.47,4,611524,9 200 zł/m2,9161-4_M13


In [ ]:
test2 = df_listings_raw[df_listings_raw['Tytul'].duplicated()]
test2

,Link,Tytul,Tytul_dzielnica,Tytul_ulica,Miasto,Dzielnica,Metraz,Pokoje,Cena,Cena_za_m2,ID
5,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Grunwald, ul. Marcelińs...",Grunwald,ul. Marcelińska 18,Poznań,Grunwald,61.77,3,Zapytajocenę,NaN,10562-B2-22
6,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Jeżyce, ul. Dąbrowskiego",Jeżyce,ul. Dąbrowskiego,Poznań,Jeżyce,83.21,4,Zapytajocenę,NaN,10115-F/8
8,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Jeżyce, ul. Dąbrowskiego",Jeżyce,ul. Dąbrowskiego,Poznań,Jeżyce,62.67,3,Zapytajocenę,NaN,10115-F/29
9,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Grunwald, ul. Marcelińs...",Grunwald,ul. Marcelińska 18,Poznań,Grunwald,87.84,4,Zapytajocenę,NaN,10562-B2-55
10,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Grunwald, ul. Marcelińs...",Grunwald,ul. Marcelińska 18,Poznań,Grunwald,62.51,3,Zapytajocenę,NaN,10562-B2-38
...,...,...,...,...,...,...,...,...,...,...,...
10,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Strzeszyn, ul. Literacka",Strzeszyn,ul. Literacka,Poznań,Strzeszyn,65.92,3,606464,9 200 zł/m2,9161-2_M15
11,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Strzeszyn, ul. Literacka",Strzeszyn,ul. Literacka,Poznań,Strzeszyn,70.43,4,647956,9 200 zł/m2,9161-1_M9
12,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Strzeszyn, ul. Literacka",Strzeszyn,ul. Literacka,Poznań,Strzeszyn,66.00,3,607200,9 200 zł/m2,9161-2_M6
13,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"Nowe mieszkanie Poznań Strzeszyn, ul. Literacka",Strzeszyn,ul. Literacka,Poznań,Strzeszyn,66.47,4,611524,9 200 zł/m2,9161-4_M13


In [ ]:
x = df_listings_raw[df_listings_raw['Tytul_ulica'] == 'ul. Wielkopolski']
x

,Link,Tytul,Tytul_dzielnica,Tytul_ulica,Miasto,Dzielnica,Metraz,Pokoje,Cena,Cena_za_m2,ID
6,https://gratka.pl/nieruchomosci/mieszkanie-poz...,"Mieszkanie Poznań Stare Miasto, ul. Wielkopolski",Stare Miasto,ul. Wielkopolski,Poznań,Stare Miasto,73.2,4,799000,10 915 zł/m2,231239


In [ ]:
ulice = df_listings_raw['Tytul_ulica'].unique()
ulice = sorted([str(i) for i in ulice])
#print(sorted(ulice))
ulice = https://www.google.com/search?q=google+maps+poznan+ul. 28 Czerwca

['',
 'nan',
 'ul. 28 Czerwca 129',
 'ul. 28 Czerwca 1956',
 'ul. 28 Czerwca 1956 r.',
 'ul. 28 Czerwca 1956/r.',
 'ul. Arnolda Szylinga',
 'ul. Berneńska',
 'ul. Bielicowa',
 'ul. Bolesława Śmiałego',
 'ul. Brneńska',
 'ul. Brzask',
 'ul. Bułgarska',
 'ul. Bułgarska 59',
 'ul. Bóżnicza',
 'ul. Ceglana',
 'ul. Ceglana 4',
 'ul. Chłapowskiego',
 'ul. Czarnucha',
 'ul. Dmowskiego',
 'ul. Dolina',
 'ul. Drewsa',
 'ul. Drogowców',
 'ul. Druskiennicka',
 'ul. Druskiennicka 2a',
 'ul. Działyńskich',
 'ul. Dąbrowskiego',
 'ul. Dąbrowskiego 58',
 'ul. Falista',
 'ul. Gabszewicza',
 'ul. Gnieźnieńska',
 'ul. Goplańska',
 'ul. Grudzieniec',
 'ul. Grunwaldzka',
 'ul. Grunwaldzka 240',
 'ul. Gwarna',
 'ul. Górczyńska',
 'ul. Górczyńska 17',
 'ul. Głogowska',
 'ul. Główna',
 'ul. Hawelańska',
 'ul. Hetmańska',
 'ul. Jackowskiego',
 'ul. Jackowskiego i Wawrzyniaka',
 'ul. Janickiego',
 'ul. Jarochowskiego 48A',
 'ul. Jarochowskiego 81',
 'ul. Jasielska',
 'ul. Jugosłowiańska',
 'ul. Kanałowa',
 'ul.

In [ ]:
address = 'ul. Marcelinska 18'
from urllib.request import urlopen
response = urlopen('https://nominatim.openstreetmap.org/search?q=' + 'Pozna%C5%84' + address.replace(' ', '+').replace('ul. ', '') +'&format=json&polygon=1&addressdetails=1')
json.load(response)
address.replace(' ', '+').replace('ul. ', '')

'ul.+Marcelinska+18'

In [ ]:
# from urllib.request import urlopen
# response = urlopen('https://nominatim.openstreetmap.org/search?q=' + 'Pozna%C5%84' + address.replace(' ', '+').replace('ul. ', '') +'&format=json&polygon=1&addressdetails=1')
# json.load(response)

# get('https://maps.googleapis.com/maps/api/geocode/json?address=1600+Amphitheatre+Parkway,+Mountain+View,+CA&key=YOUR_API_KEY')
# address = 'Wilda, ul. 28 Czerwca 1956 r.'
# j = json.load(urllib.request.urlopen('https://nominatim.openstreetmap.org/search?q=' + 'Pozna%C5%84' + address +'&format=json&polygon=1&addressdetails=1'))
# "addresstype":"suburb"

#Google geocoding API
# https://console.cloud.google.com/projectselector2/google/maps-apis/credentials?utm_source=Docs_CreateAPIKey&utm_content=Docs_geocoding-backend&hl=pl&pli=1&authuser=1

#Check if the coordinates are within boundary shape
# https://gis.stackexchange.com/questions/208546/check-if-a-point-falls-within-a-multipolygon-with-python

#Probably even with this package
# https://streamhacker.com/2010/03/23/python-point-in-polygon-shapely/

#Or just a circle
# https://stackoverflow.com/questions/481144/equation-for-testing-if-a-point-is-inside-a-circle

##Get the screenshot to check the current status

In [ ]:
# Set the window size to match the entire page
page_height = wd.execute_script("return Math.max( document.body.scrollHeight, document.body.offsetHeight, document.documentElement.clientHeight, document.documentElement.scrollHeight, document.documentElement.offsetHeight );")
wd.set_window_size(wd.get_window_size()['width'], page_height)

#or

# wd.manage().window().maximize()

wd.save_screenshot("screenshot.png")

True

# Checking if the street is of an interest

https://eteryt.stat.gov.pl/eTeryt/rejestr_teryt/udostepnianie_danych/baza_teryt/uzytkownicy_indywidualni/wyszukiwanie/wyszukiwanie.aspx?contrast=default

lub

https://eteryt.stat.gov.pl/eTeryt/rejestr_teryt/udostepnianie_danych/baza_teryt/uzytkownicy_indywidualni/przegladanie/przegladanie.aspx?contrast=default